**Open Items:**

Holiday
    Flag for the specific category (day before, Day After) - DONE

*Mean, Std for each of the below:*

Day of week - DONE

Area - DONE

Area day of week - DONE
    
Genre - DONE

Genre day of week - DONE

Week of month - DO WE WANT TO DO THIS 1-7 OR FIRST MONDAY?
    
Days in business - DO WE NEED THIS?

*Other Notes:*

Could use last year visitor numbers for baseline to compare model performance with

In [1]:
import pandas as pd

In [51]:
rest_all = pd.read_csv("full_1120.csv")

In [52]:
def feature_eng(input_df):
    
    # Create temp data frame to work with - Consider only rows with stores open for visitors
    temp_df = input_df[input_df["closed_flag"] == 0]
    
    # Day of week
    dayofweek_group = temp_df.groupby("dayofweek").agg({"visitors":["mean","std"]})
    output_df = input_df.merge(dayofweek_group, left_on = "dayofweek", right_index = True)
    output_df["dayofweek_mean"] = output_df[('visitors','mean')]
    output_df["dayofweek_std"] = output_df[('visitors','std')]
    output_df = output_df.drop([('visitors','std'),('visitors','mean')], axis=1)
    
    # Genre
    genre_group = temp_df.groupby("air_genre_name").agg({"visitors":["mean","std"]})
    output_df = output_df.merge(genre_group, left_on = "air_genre_name", right_index = True)
    output_df["genre_mean"] = output_df[('visitors','mean')]
    output_df["genre_std"] = output_df[('visitors','std')]
    output_df = output_df.drop([('visitors','std'),('visitors','mean')], axis=1)
    
    # Genre day of week
    genre_dayofweek_group = rest_all.groupby(["air_genre_name","dayofweek"]).agg({"visitors":["mean","std"]})
    output_df = output_df.merge(genre_dayofweek_group, left_on = ["air_genre_name","dayofweek"], right_index = True)
    output_df["genre_dayofweek_mean"] = output_df[('visitors','mean')]
    output_df["genre_dayof_week_std"] = output_df[('visitors','std')]
    output_df = output_df.drop([('visitors','std'),('visitors','mean')], axis=1)
    
    # Area
    area_group = temp_df.groupby("air_area_name").agg({"visitors":["mean","std"]})
    output_df = output_df.merge(area_group, left_on = "air_area_name", right_index = True)
    output_df["area_mean"] = output_df[('visitors','mean')]
    output_df["area_std"] = output_df[('visitors','std')]
    output_df = output_df.drop([('visitors','std'),('visitors','mean')], axis=1)
    
    # Area day of week
    area_dayofweek_group = temp_df.groupby(["air_area_name","dayofweek"]).agg({"visitors":["mean","std"]})
    output_df = output_df.merge(area_dayofweek_group, left_on = ["air_area_name","dayofweek"], right_index = True)
    output_df["area_dayofweek_mean"] = output_df[('visitors','mean')]
    output_df["area_dayof_week_std"] = output_df[('visitors','std')]
    output_df = output_df.drop([('visitors','std'),('visitors','mean')], axis=1)
    
    # Day before / after holidays
    # For consecutive holdays: do not allow holidays to have the day after or day before flag
    # Does not handle situations where day after or before is a "closed" day
    output_df['day_after_holiday_flg'] = output_df['holiday_flg'].shift(1)
    output_df['day_before_holiday_flg'] = output_df['holiday_flg'].shift(-1)
    output_df.loc[output_df["holiday_flg"] == 1, ["day_before_holiday_flg", 'day_after_holiday_flg']] = 0
    output_df["day_before_holiday_flg"].fillna(0, inplace=True)
    output_df['day_after_holiday_flg'].fillna(0, inplace=True)
    
    return output_df

feature_eng(rest_all)

C:\Users\Patrick Seng\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,reserve_visitors_air,hpg_store_id,...,genre_mean,genre_std,genre_dayofweek_mean,genre_dayof_week_std,area_mean,area_std,area_dayofweek_mean,area_dayof_week_std,day_after_holiday_flg,day_before_holiday_flg
0,air_ba937bf13d40fb24,2016-01-01,Friday,1,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,18.723532,16.00068,21.703760,16.326418,19.609418,15.750431,23.509685,16.870727,0.0,0.0
7,air_ba937bf13d40fb24,2016-01-08,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,18.723532,16.00068,21.703760,16.326418,19.609418,15.750431,23.509685,16.870727,1.0,0.0
14,air_ba937bf13d40fb24,2016-01-15,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,18.723532,16.00068,21.703760,16.326418,19.609418,15.750431,23.509685,16.870727,0.0,0.0
21,air_ba937bf13d40fb24,2016-01-22,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,18.723532,16.00068,21.703760,16.326418,19.609418,15.750431,23.509685,16.870727,0.0,0.0
28,air_ba937bf13d40fb24,2016-01-29,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,18.723532,16.00068,21.703760,16.326418,19.609418,15.750431,23.509685,16.870727,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372722,air_dfad598ff642dab7,2017-04-27,Thursday,0,Yakiniku/Korean food,Hokkaidō Abashiri-shi Minami 6 Jōhigashi,44.020632,144.273398,NaN,NaN,...,21.235872,14.53503,19.364319,14.040779,28.844828,14.368568,24.580153,14.354819,0.0,1.0
372729,air_dfad598ff642dab7,2017-05-04,Thursday,1,Yakiniku/Korean food,Hokkaidō Abashiri-shi Minami 6 Jōhigashi,44.020632,144.273398,NaN,NaN,...,21.235872,14.53503,19.364319,14.040779,28.844828,14.368568,24.580153,14.354819,0.0,0.0
372736,air_dfad598ff642dab7,2017-05-11,Thursday,0,Yakiniku/Korean food,Hokkaidō Abashiri-shi Minami 6 Jōhigashi,44.020632,144.273398,NaN,NaN,...,21.235872,14.53503,19.364319,14.040779,28.844828,14.368568,24.580153,14.354819,1.0,0.0
372743,air_dfad598ff642dab7,2017-05-18,Thursday,0,Yakiniku/Korean food,Hokkaidō Abashiri-shi Minami 6 Jōhigashi,44.020632,144.273398,NaN,NaN,...,21.235872,14.53503,19.364319,14.040779,28.844828,14.368568,24.580153,14.354819,0.0,0.0
